In [17]:
import networkx as nx

# Replace 'graph.adjlist' with the path to your adjacency list file
G = nx.read_adjlist('graph1.adjlist')
betweenness_centrality = nx.betweenness_centrality(G, normalized=False)
mx = max(betweenness_centrality.values())

In [18]:
print(mx)

3231.036299718778


In [20]:
def brandes_betweenness_centrality(graph, normalized=False):
    """
    Calculate the betweenness centrality of a NetworkX graph using Brandes' algorithm.

    Parameters:
        graph (networkx.Graph): The input graph.
        normalized (bool): Whether to normalize the centrality values. Default is True.

    Returns:
        dict: A dictionary where keys are nodes and values are their betweenness centrality.
    """
    betweenness = dict.fromkeys(graph, 0.0)  # Initialize betweenness centrality for all nodes
    for s in graph:
        # Single-source shortest paths
        stack = []
        pred = {w: [] for w in graph}  # Predecessors
        sigma = dict.fromkeys(graph, 0.0)  # Number of shortest paths
        sigma[s] = 1.0
        dist = dict.fromkeys(graph, -1)  # Distance from source
        dist[s] = 0
        queue = [s]
        
        while queue:
            v = queue.pop(0)
            stack.append(v)
            for w in graph[v]:
                # Path discovery
                if dist[w] == -1:  # w found for the first time
                    dist[w] = dist[v] + 1
                    queue.append(w)
                # Path counting
                if dist[w] == dist[v] + 1:  # Is w on the shortest path?
                    sigma[w] += sigma[v]
                    pred[w].append(v)
        
        # Accumulation
        delta = dict.fromkeys(graph, 0.0)  # Dependency
        while stack:
            w = stack.pop()
            for v in pred[w]:
                delta[v] += (sigma[v] / sigma[w]) * (1 + delta[w])
            if w != s:
                betweenness[w] += delta[w]
    
    # Normalization
    if normalized:
        scale = 1 / ((len(graph) - 1) * (len(graph) - 2)) if len(graph) > 2 else None
        if scale is not None:
            for v in betweenness:
                betweenness[v] *= scale
    else:
        for v in betweenness:
            betweenness[v] /= 2.0  # For undirected graphs
    
    return betweenness

In [22]:
print(max(brandes_betweenness_centrality(G, normalized=False).values()))

3231.0362997187776
